# Install dependencies

In [6]:
# !pip install Keras==2.1.6 pydot tensorflow-gpu==1.8.0 
# !sudo apt-get install graphviz -y

    100% |████████████████████████████████| 8.9MB 4.3MB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 14.3MB/s ta 0:00:01
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Imports

In [1]:
from keras.utils.vis_utils import model_to_dot
import pydot
from IPython.display import SVG
import json
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.utils import multi_gpu_model
from keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
import pandas as pd
from sklearn.metrics import confusion_matrix
import itertools
import shutil
import os
import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


# Setup

In [3]:
!ls data/images

apple_pie	    eggs_benedict	     onion_rings
baby_back_ribs	    escargots		     oysters
baklava		    falafel		     pad_thai
beef_carpaccio	    filet_mignon	     paella
beef_tartare	    fish_and_chips	     pancakes
beet_salad	    foie_gras		     panna_cotta
beignets	    french_fries	     peking_duck
bibimbap	    french_onion_soup	     pho
bread_pudding	    french_toast	     pizza
breakfast_burrito   fried_calamari	     pork_chop
bruschetta	    fried_rice		     poutine
caesar_salad	    frozen_yogurt	     prime_rib
cannoli		    garlic_bread	     pulled_pork_sandwich
caprese_salad	    gnocchi		     ramen
carrot_cake	    greek_salad		     ravioli
ceviche		    grilled_cheese_sandwich  red_velvet_cake
cheesecake	    grilled_salmon	     risotto
cheese_plate	    guacamole		     samosa
chicken_curry	    gyoza		     sashimi
chicken_quesadilla  hamburger		     scallops
chicken_wings	    hot_and_sour_soup	     seaweed_salad
chocolate_cake	    hot_dog		     shrimp_and_grits
chocolate_mousse    huevo

In [14]:
NUM_GPUS = 1
train_folder = "data/train"
valid_folder = "data/valid"

In [15]:
!ls data/

images


In [16]:
IMAGE_SIZE=224 
BATCH_SIZE=64 # number of images seen by model at once

In [4]:
!ls data/train/ | head

ls: cannot access 'data/train/': No such file or directory


In [ ]:
train_generator = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input
                                     ,rotation_range=45
                                     ,width_shift_range=0.2
                                     ,height_shift_range=0.2
                                     ,shear_range=0.2
                                     ,zoom_range=0.25
                                     ,horizontal_flip=True
                                     ,fill_mode='nearest'
                                  )

train_batches = train_generator.flow_from_directory(
    train_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE,shuffle=True, seed=13,class_mode='categorical')



valid_generator =  ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)


valid_batches = valid_generator.flow_from_directory(
    valid_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE)

In [ ]:
NUM_CLASSES = train_batches.num_classes

In [ ]:
base_model = keras.applications.mobilenet.MobileNet()

In [ ]:
?keras.applications.mobilenet.MobileNet()

In [ ]:
base_model.summary()

In [ ]:
SVG(model_to_dot(base_model,show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))

In [ ]:
base_model.layers[-6].output

In [ ]:
x = base_model.layers[-6].output
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.summary()

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
SVG(model_to_dot(model,show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))

In [ ]:
model.summary()

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9)

#optimizer = Adam(lr=0.001)

# model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

#model.compile(Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
?multi_gpu_model

In [ ]:
try:
    model = multi_gpu_model(model, gpus=NUM_GPUS)
except:
    pass
model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
steps_per_epoch = train_batches.n//BATCH_SIZE
validation_steps =valid_batches.n // BATCH_SIZE

# steps_per_epoch = 10
# validation_steps = 3

In [ ]:
!rm -rf logs

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1)

tb = keras.callbacks.TensorBoard(log_dir=f'{PATH}/logs', write_graph=True, write_images=True)


In [ ]:
model.fit_generator(train_batches, steps_per_epoch=steps_per_epoch 
                    , validation_data=valid_batches
                    ,  validation_steps=validation_steps, verbose=2,workers=4 
                    , use_multiprocessing=True
                    , epochs=3
                   # , callbacks = [tb]
                   # , callbacks = [reduce_lr, tb]
                   )

In [ ]:
train_batches.class_indices.keys()

In [ ]:
train_batches.class_indices

In [ ]:
MODEL_SAVE_DIR = f'model/{DATASET_NAME}'

In [ ]:
!mkdir -p {MODEL_SAVE_DIR}

In [ ]:
shutil.rmtree(MODEL_SAVE_DIR)
os.makedirs(MODEL_SAVE_DIR)

In [ ]:
model.save(f'{MODEL_SAVE_DIR}/model.h5')

In [ ]:
with open(f'{MODEL_SAVE_DIR}/labels.txt', 'w') as file_handler:
    for item in train_batches.class_indices.keys():
        file_handler.write("{}\n".format(item))

In [ ]:
with open(f'{MODEL_SAVE_DIR}/labels.json', 'w') as outfile:
    json.dump(train_batches.class_indices, outfile)